In [11]:
"""
OptiGaussPy
==========
Herramienta interactiva para graficar pulsos gaussianos con chirp y dispersión cromática
en función de parámetros del Tx óptico, características y longitud de la fibra óptica.
"""
import numpy as np
from scipy import signal, constants
from scipy.signal import chirp, spectrogram
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pint

ureg = pint.UnitRegistry()
pint._DEFAULT_REGISTRY = ureg
c_luz = constants.c * (ureg.meter) / (1 * ureg.second)
pi = constants.pi

In [12]:
def modulo(numero):
    """"
    Funcion auxiliar que calcula el modulo de un numero
    ===================================================
    recibe una variable de tipo pint.quantity
    retorna su módulo conservando sus unidades científicas
    """
    if numero.magnitude < 0:
        return -1 * numero
    else:
        return numero

In [13]:
def calculo_ensanchamiento_temporal_inicial(bitrate):
    """
    Calculo del ensalchamiento temporal inicial de un pulso gaussiano
    =================================================================
    Retorna una variable del tipo pint.quantity en 'second'
    Recibe el valor de bitrate del Tx Óptico
    en una variable del tipo pint.quantity en 'gigabit/second'
    tomando como ensanchamiento temporal inicial
    mitad de tiempo de bit
    """
    bps = bitrate.to_base_units() #bps
    tiempo_de_bit = 1 / (bps/ureg.bit) #s
    T_o = tiempo_de_bit / 2 #s
    return T_o

In [14]:
def calculo_beta2(lambda_o, dispersion_cromatica):
    """
    Calculo de β2
    ================
    Retorna una variable del tipo pint.quantity en 'picosecond ** 2 / kilometer'
    recibe dos variables del tipo pint.quantity
    lambda_o (λo) en 'nanometer'
    dispersion_cromatica (D(λ)) en 'picosecond / (nanometer * kilometer)'
    calcula β2 según la siguiente expresión:
    $\beta_2 = -\frac{\lambda^2}{2\pi c}D$
    """
    numerador = -( (lambda_o.to_base_units() **2) * dispersion_cromatica.to_base_units() )
    denominador = (2 * pi * c_luz)
    beta_2 = numerador / denominador
    return beta_2

In [15]:
def calculo_ensanchamiento_temporal(chirp_o, longitud, T_o, beta_2):
    """
    Calculo ensanchamiento temporal
    ===============================
    Retorna dos variables del tipo pint.quantity 
    T_z, ensanchamiento temporal en 'second'
    Tz_To, relación entre ensanchamientos temporales 'dimensionless'
    Recibe tres variables del tipo pint.quantity
    chirp_o, chirp inicial 'dimensionless'
    longitud de la fibra en 'kilometer'
    T_o, ensanchamiento temporal inicial en 'second'
    beta_2 en 'picosecond ** 2 / kilometer'
    #"$\frac{T_z}{T_o} = \sqrt{(1 + \frac{\beta_2zk_o}{To^2})^2+(\frac{\beta_2z}{T_o^2})^2}$"#
    """
    A = (beta_2.to_base_units() * longitud.to_base_units() * chirp_o) / (T_o.to_base_units()**2)
    B = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o.to_base_units()**2)
    T_z = ( ( ((1 + A)**2) + B**2 ) ** (1/2) ) * T_o
    Tz_To = T_z / T_o
    return T_z, Tz_To

In [16]:
def calculo_chirp(T_o, chirp_o, longitud, beta_2):
    """
    Calculo chirp
    =============
    Retorna una variable del tipo pint.quantity 'dimensionless'
    con el valor del chirp del pulso gaussiano al cabo de longitud
    Recibe cuatro variables del tipo pint.quantity
    T_o, Ensanchamiento temporal inicial en 'second'
    chirp_o, Chirp inicial 'dimensionless'
    longitud, de la fibra en 'kilometer'
    beta_2 en 'picosecond ** 2 / kilometer'
    Calcula el chirp según la siguiente expresión:
    "$k_z = k_o + \frac{(1+k_o^2)\beta_2z}{T_0^2}$"
    #A = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o**2).to_base_units()
    #chirp_z = chirp_o * (1 + chirp_o**2) + A
    """
    if longitud == (0 * ureg('kilometer')) :
        chirp_z = chirp_o
    else: 
        chirp_z = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o**2).to_base_units() + chirp_o * (1 + chirp_o**2)
    
    return chirp_z

In [17]:
def calculo_longitud_dispersion(T_o, beta_2):
    """
    Calculo longitud de dispersion
    ==============================
    Retorna una variable del tipo pint.quantity
    L_Dispersion en 'kilometer'
    Recibe dos variables del tipo pint.quantity
    T_o, ensanchamiento temporal inicial en 'second'
    beta_2 en 'picosecond ** 2 / kilometer'
    Se calcula la longitud de dispersión segun la siguiente expresión:
    $L_D = \frac{2T_o}{\beta_2}$
    """
    if beta_2.magnitude == 0 :
        L_Dispersion = float('inf') * ureg('kilometer')
    else:
        mod_beta2 = (modulo(beta_2))
        L_Dispersion = ( (T_o**2) / (mod_beta2) ).to(ureg('kilometer'))
        
    return L_Dispersion

In [18]:
def calculo_parametros(bitrate, lambda_o , dispersion_cromatica, chirp_o, longitud):
    """
    Calculo de parametros
    =====================
    Retorna 6 variables del tipo pint.quantity
    T_o, ensanchamiento temporal inicial en 'second'
    T_z, ensanchamiento temporal en 'second'
    Tz_To, relación entre ensanchamientos temporales 'dimensionless'
    beta_2 en 'picosecond ** 2 / kilometer'
    chirp_z, chirp del pulso gaussiano al cabo de longitud 'dimensionless'
    L_Dispersion en 'kilometer'
    Recibe 5 variables del tipo pint.quantity
    bitrate, del tipo pint.quantity en 'gigabit/second'
    (λo) en 'nanometer'
    dispersion_cromatica (D(λ)) en 'picosecond / (nanometer * kilometer)'
    chirp_o, chirp inicial 'dimensionless'
    longitud de la fibra en 'kilometer'
    ==================
    Funciones internas
    =======================================
    calculo_ensanchamiento_temporal_inicial
    calculo_ensanchamiento_temporal
    calculo_beta2
    calculo_chirp
    calculo_longitud_dispersion
    =======================================
    """
    T_o = calculo_ensanchamiento_temporal_inicial(bitrate)
    beta_2 = calculo_beta2(lambda_o, dispersion_cromatica)
    T_z, Tz_To = calculo_ensanchamiento_temporal(chirp_o, longitud, T_o, beta_2)
    chirp_z = calculo_chirp(T_o, chirp_o, longitud, beta_2)
    L_Dispersion = calculo_longitud_dispersion(T_o, beta_2)
    return T_o, beta_2, T_z, Tz_To, chirp_z, L_Dispersion